In [1]:
import pandas as pd
from great_tables import GT, md, vals, nanoplot_options, style, loc

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
who_tb_data = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-11-11/who_tb_data.csv")

In [3]:
who_tb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                5117 non-null   object 
 1   g_whoregion            5117 non-null   object 
 2   iso_numeric            5117 non-null   int64  
 3   iso2                   5117 non-null   object 
 4   iso3                   5117 non-null   object 
 5   year                   5117 non-null   int64  
 6   c_cdr                  4824 non-null   float64
 7   c_newinc_100k          4932 non-null   float64
 8   cfr                    4989 non-null   float64
 9   e_inc_100k             5117 non-null   float64
 10  e_inc_num              5117 non-null   int64  
 11  e_mort_100k            5093 non-null   float64
 12  e_mort_exc_tbhiv_100k  5093 non-null   float64
 13  e_mort_exc_tbhiv_num   5093 non-null   float64
 14  e_mort_num             5093 non-null   float64
 15  e_mo

In [4]:
who_tb_data.head(2)

,country,g_whoregion,iso_numeric,iso2,iso3,year,c_cdr,c_newinc_100k,cfr,e_inc_100k,e_inc_num,e_mort_100k,e_mort_exc_tbhiv_100k,e_mort_exc_tbhiv_num,e_mort_num,e_mort_tbhiv_100k,e_mort_tbhiv_num,e_pop_num
0,Afghanistan,Eastern Mediterranean,4,AF,AFG,2000,19.0,35.0,0.37,190.0,38000,68.0,68.0,14000.0,14000.0,0.17,34.0,20130323
1,Afghanistan,Eastern Mediterranean,4,AF,AFG,2001,26.0,50.0,0.35,189.0,38000,63.0,63.0,13000.0,13000.0,0.30,61.0,20284311


In [5]:
# Select relevant columns
sel_col = who_tb_data[["g_whoregion", "year", "e_inc_100k", "e_mort_exc_tbhiv_100k", "e_mort_tbhiv_100k"]]

# 2020 vs 2023 data prepartion

In [6]:
# Filter relevant years
years = [2000, 2023]
df_years1=sel_col[sel_col["year"].isin(years)]

In [7]:
# Mean values per region and year
df_years2=df_years1.groupby(["g_whoregion", "year"]).mean().reset_index()

# Calculate 2020 vs 2023 YoY change 
df_years2["e_inc_100k_YoY_change"] = df_years2.groupby("g_whoregion")["e_inc_100k"].pct_change(fill_method=None) 
df_years2["e_mort_exc_tbhiv_100k_YoY_change"] = df_years2.groupby("g_whoregion")["e_mort_exc_tbhiv_100k"].pct_change(fill_method=None)  
df_years2["e_mort_tbhiv_100k_YoY_change"] = df_years2.groupby("g_whoregion")["e_mort_tbhiv_100k"].pct_change(fill_method=None) 

In [8]:
# Order the columns
df_years3 = df_years2[["g_whoregion", "year",                       
                       "e_inc_100k", "e_inc_100k_YoY_change",
                       "e_mort_exc_tbhiv_100k", "e_mort_exc_tbhiv_100k_YoY_change",
                       "e_mort_tbhiv_100k", "e_mort_tbhiv_100k_YoY_change"]]                

In [9]:
# Pivot year values into columns
df_years4=df_years3.groupby(["g_whoregion", "year"]).mean().unstack() 
# Drop Na columns
df_years5=df_years4.dropna(axis=1)

In [10]:
# Convert column names to a df
df_reset = df_years5.columns.to_frame(index=False).reset_index(drop=True)

# Rename the columns by joining their names with underscores
df_years5.columns = ['_'.join(map(str, col)) for col in df_years5.columns]

# Reset index
df_years6 = df_years5.reset_index()

In [11]:
# Create empty columns
df_years6["e_inc_100k_sym"] = ""
df_years6["e_mort_exc_tbhiv_100k_sym"] = ""
df_years6["e_mort_tbhiv_100k_sym"] = ""

# Function to add "▲" or "▼"
def add_symbol(val):
    if val > 0:
        return "▲"
    else :
        return "▼"
    
# Apply the function 
df_years6["e_inc_100k_sym"] = df_years6["e_inc_100k_YoY_change_2023"].apply(add_symbol)
df_years6["e_mort_exc_tbhiv_100k_sym"] = df_years6["e_mort_exc_tbhiv_100k_YoY_change_2023"].apply(add_symbol)
df_years6["e_mort_tbhiv_100k_sym"] = df_years6["e_mort_tbhiv_100k_YoY_change_2023"].apply(add_symbol)

In [12]:
# Order the columns
df_years7 = df_years6[["g_whoregion", 
                       "e_inc_100k_2000", "e_inc_100k_2023", "e_inc_100k_sym", "e_inc_100k_YoY_change_2023", 
                       "e_mort_exc_tbhiv_100k_2000", "e_mort_exc_tbhiv_100k_2023", "e_mort_exc_tbhiv_100k_sym","e_mort_exc_tbhiv_100k_YoY_change_2023", 
                       "e_mort_tbhiv_100k_2000", "e_mort_tbhiv_100k_2023", "e_mort_tbhiv_100k_sym", "e_mort_tbhiv_100k_YoY_change_2023"]]

# Nanoplot data preparation

In [13]:
# Calculate the mean of numeric columns
df_lists1=sel_col.groupby(["g_whoregion", "year"]).mean(numeric_only=True).reset_index()

In [14]:
# Aggregate columns into lists
df_lists2 = df_lists1.groupby("g_whoregion").agg(list).drop(columns="year").reset_index()
#df_lists2 = df_lists2.drop("year", axis=1)

In [15]:
# Convert list columns to space separated strings 
for col in [c for c in df_lists2.columns]:
    df_lists2[col] = df_lists2[col].apply(lambda x: " ".join(str(v) for v in x))

# Drop region column
df_lists3=df_lists2.drop("g_whoregion", axis=1)

# Concatenation

In [16]:
# Concat dataframes horizontally
df=pd.concat([df_years7, df_lists3], axis=1)

In [17]:
# Order the columns
df = df[["g_whoregion", 
         "e_inc_100k_2000", "e_inc_100k_2023", "e_inc_100k_sym", "e_inc_100k_YoY_change_2023", "e_inc_100k",
         "e_mort_exc_tbhiv_100k_2000", "e_mort_exc_tbhiv_100k_2023", "e_mort_exc_tbhiv_100k_sym","e_mort_exc_tbhiv_100k_YoY_change_2023", "e_mort_exc_tbhiv_100k",
         "e_mort_tbhiv_100k_2000", "e_mort_tbhiv_100k_2023", "e_mort_tbhiv_100k_sym", "e_mort_tbhiv_100k_YoY_change_2023", "e_mort_tbhiv_100k"]]

# Final df

In [18]:
# Split dataframes by type
df1 = df.iloc[:, :6]
df2 = pd.concat([df.iloc[:, 0], df.iloc[:, 6:11]], axis=1)
df3 = pd.concat([df.iloc[:, 0], df.iloc[:, 11:16]], axis=1)

In [19]:
# Create new column and renames the other columns
df1["type"] = "Incidence per 100 000 population"
df1.columns = ["region", "y_2000", "y_2023", "sym", "change", "trend", "type"]

df2["type"] = "Non-HIV mortality per 100 000 population"
df2.columns = ["region", "y_2000", "y_2023", "sym", "change", "trend", "type"]

df3["type"] = "HIV mortality per 100 000 population"
df3.columns = ["region", "y_2000", "y_2023", "sym", "change", "trend", "type"]

In [20]:
# Concatenate 3 dataframes
result = pd.concat([df1, df2, df3], ignore_index=True)

# Great tables

In [21]:
# Reset the index to include it as a column
result = result.reset_index()  

table = (
    GT(result)
    .tab_header(
        title=md("Global tuberculosis (TB) burden estimates")        
    )     
    .tab_stub(
        rowname_col="region", groupname_col="type"
    )      
    .cols_label(
        index="Index",  
        y_2000="2000",
        y_2023="2023",
        change="Change",
        sym="(%)",          
        trend="Trend (2000-2023)",        
    )    
    .fmt_number(
        columns=["y_2000", "y_2023"], decimals=1, use_seps=False
    )
    .fmt_percent(
        "change", decimals=0
    )  
    .cols_move(
        columns="sym",
        after="change"
    )
    .cols_align(
        align="center"
    )  
    .cols_align(
        align="left", columns="sym"
    )          
    .cols_align(
        align="right", columns="change"
    )
    .cols_hide(
        columns="index"
    )   
    .fmt_nanoplot(
        columns="trend", plot_type="bar",
        options=nanoplot_options(            
            data_bar_fill_color="#093362",
        ),
    )     
    .tab_source_note(
        source_note="Data: TidyTuesday 2025-11-11 (WHO TB Burden Data: Incidence, Mortality, and Population) | Design: Bernadett Piros"
    )  
    .tab_style(
        style=style.text(size="x-small", align="center"),            
        locations=loc.source_notes()  
    )
    .tab_style(
        style=style.text(size="x-large", color="#093362", weight="bold"),            
        locations=loc.title()  
    )
    .tab_style(
        style=style.text(color="#093362", weight="bold"),            
        locations=loc.column_labels() 
    )    
    .tab_options(             
        row_group_font_weight="bold",
        row_group_background_color="#E0C954"
    )
    .tab_style(
        style=[
            style.fill(color="#FFEB7E"),  # darker color
            style.borders(sides=["top", "bottom", "left", "top"], weight="0px")
        ],
        locations=loc.stub(rows=lambda D: D['index'] % 2 == 0)  # Even rows
    )
    .tab_style(
        style=[
            style.fill(color="#FFF09A"),  # lighter color
            style.borders(sides=["top", "bottom", "left", "top"], weight="0px")
        ],
        locations=loc.stub(rows=lambda D: D['index'] % 2 != 0)  # Odd rows
    )
     .tab_style(
        style=[
            style.fill(color="#FFEB7E"),  # darker color
            style.borders(sides=["top", "bottom", "left", "top"], weight="0px")
        ],
        locations=loc.body(rows=lambda D: D['index'] % 2 == 0)  # Even rows
    )
    .tab_style(
        style=[
            style.fill(color="#FFF09A"),  # lighter color
            style.borders(sides=["top", "bottom", "left", "top"], weight="0px")
        ],
        locations=loc.body(rows=lambda D: D['index'] % 2 != 0)  # Odd rows
    )
    .tab_style(
        style=style.text(color="#FD4357"),   
        locations=loc.body(rows=lambda D: D["sym"] == "▲", columns="sym")
    )
    .tab_style(
        style=style.text(color="#049C3F"),   
        locations=loc.body(rows=lambda D: D["sym"] == "▼", columns="sym")  
    )
    .tab_style(
        style=style.fill(color="#BF9106"),            
        locations=loc.column_labels() 
    )   
    .tab_style(
        style=style.fill(color="#BF9106"),            
        locations=loc.stubhead() 
    )    
    .tab_style(
        style=style.fill(color="#BF9106"),            
        locations=loc.title() 
    )    
    .tab_style(
        style=style.fill(color="#FDF5DC"),            
        locations=loc.source_notes() 
    )    
    .opt_vertical_padding(
        scale=0.5
    )
    .tab_options(             
        row_group_border_top_color="#E0C954",
        row_group_border_bottom_color="#E0C954",  
        table_border_top_color="#BF9106",
        table_border_bottom_color="#FDF5DC", 
        column_labels_border_top_color="#E0C954",
        stub_border_color="#E0C954",
        table_body_border_bottom_color="#E0C954",
    )    
)
table

GT(_tbl_data=    index                 region      y_2000      y_2023 sym    change  \
0       0                 Africa  327.108696  204.702128   ▼ -0.374208   
1       1               Americas   39.290698   30.764889   ▼ -0.216993   
2       2  Eastern Mediterranean   99.163636   61.538636   ▼ -0.379423   
3       3                 Europe   52.113462   23.199815   ▼ -0.554821   
4       4        South-East Asia  292.555556  263.700000   ▼ -0.098633   
5       5        Western Pacific  137.208108  147.716216   ▲  0.076585   
6       6                 Africa   39.931304   24.107660   ▼ -0.396272   
7       7               Americas    4.285116    2.570222   ▼ -0.400198   
8       8  Eastern Mediterranean   15.064091    7.942273   ▼ -0.472768   
9       9                 Europe    6.205577    1.574074   ▼ -0.746345   
10     10        South-East Asia   64.787500   30.122222   ▼ -0.535061   
11     11        Western Pacific   17.025676   12.504054   ▼ -0.265577   
12     12                 Africa   73.240435   17.241277   ▼ -0.764593   
13     13               Americas    1.949535    0.863111   ▼ -0.557273   
14     14  Eastern Mediterranean    3.298182    0.230455   ▼ -0.930127   
15     15                 Europe    0.352308    0.511667   ▲  0.452329   
16     16        South-East Asia    8.198750    1.694444   ▼ -0.793329   
17     17        Western Pacific    0.747568    0.484865   ▼ -0.351410   

                                                trend  \
0   327.10869565217394 334.32608695652175 341.2391...   
1   39.2906976744186 37.95232558139535 37.01627906...   
2   99.16363636363636 98.09545454545454 89.3136363...   
3   52.11346153846154 52.07115384615384 50.9846153...   
4   292.55555555555554 288.77777777777777 308.0 30...   
5   137.2081081081081 133.5810810810811 140.129729...   
6   39.931304347826085 42.18478260869565 42.512391...   
7   4.285116279069768 4.065581395348837 3.92674418...   
8   15.06409090909091 14.485909090909091 13.668181...   
9   6.205576923076923 6.114423076923076 5.81499999...   
10  64.7875 62.65 64.03333333333333 58.06666666666...   
11  17.025675675675675 16.303783783783786 16.81864...   
12  73.24043478260869 72.44717391304349 68.0454347...   
13  1.94953488372093 1.764186046511628 1.496976744...   
14  3.2981818181818183 3.0868181818181815 2.235 2....   
15  0.3523076923076923 0.32865384615384613 0.32057...   
16  8.19875 8.216249999999999 6.884444444444445 6....   
17  0.7475675675675676 0.8102702702702703 1.284864...   

                                        type  
0           Incidence per 100 000 population  
1           Incidence per 100 000 population  
2           Incidence per 100 000 population  
3           Incidence per 100 000 population  
4           Incidence per 100 000 population  
5           Incidence per 100 000 population  
6   Non-HIV mortality per 100 000 population  
7   Non-HIV mortality per 100 000 population  
8   Non-HIV mortality per 100 000 population  
9   Non-HIV mortality per 100 000 population  
10  Non-HIV mortality per 100 000 population  
11  Non-HIV mortality per 100 000 population  
12      HIV mortality per 100 000 population  
13      HIV mortality per 100 000 population  
14      HIV mortality per 100 000 population  
15      HIV mortality per 100 000 population  
16      HIV mortality per 100 000 population  
17      HIV mortality per 100 000 population  , _body=<great_tables._gt_data.Body object at 0x7b95984c5910>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.hidden: 4>, column_label='Index', column_align='center', column_width=None), ColInfo(var='region', type=<ColInfoTypeEnum.stub: 2>, column_label='region', column_align='center', column_width=None), ColInfo(var='y_2000', type=<ColInfoTypeEnum.default: 1>, column_label='2000', column_align='center', column_width=None), ColInfo(var='y_2023', type=<ColInfoTypeEnum.default: 1>, column_label='2023', column_align='center', column_width=None), ColInfo(var='change', type=<ColInfoTypeEn

# Save html without selenium
html = table.as_raw_html()
with open("2023-11-11.html", "w") as f:
    f.write(html)